In [264]:
import numpy as np 
import pandas as pd 
from keras.models import Sequential
from keras.layers import Activation, Dropout
from keras.optimizers import SGD
from keras.constraints import max_norm
from sklearn.linear_model import LogisticRegression
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import Dense
from sklearn import preprocessing
path = '../../link_to_data_for_foundataion_of_ds/5000vocab_joined_comments'

In [265]:
train = pd.read_csv(path + '/training_data_mbti_one_comment.csv')
test = pd.read_csv(path + '/testing_data_mbti_one_comment.csv')

In [266]:
print(train.head())

   Unnamed: 0  type  average_word_length  stop_word_%  upper_word_% t1 t2 t3  \
0         893  INTP                 4.46         0.35          0.05  I  N  T   
1        1425  INFP                 4.37         0.40          0.04  I  N  F   
2        2791  ENTP                 4.55         0.40          0.05  E  N  T   
3        1205  INFJ                 4.50         0.31          0.05  I  N  F   
4        7588  INTJ                 4.26         0.37          0.04  I  N  T   

  t4  abandon  ...   youth  youtube  youve  yr  yup  zen  zero  zodiac  \
0  P        0  ...       0        0      0   0    0    0     0       0   
1  P        0  ...       0        0      0   0    0    0     0       0   
2  P        0  ...       0        0      2   0    0    0     0       0   
3  J        0  ...       0        0      0   0    0    0     0       0   
4  J        0  ...       0        0      0   1    0    0     0       0   

   zombie  zone  
0       0     0  
1       0     0  
2       0     0  
3 

In [267]:
f_type = train['type']
t1 = train['t1']
t2 = train['t2']
t3 = train['t3']
t4 = train['t4']
train = train.drop(['t1','t2', 't3', 't4', 'user', 'type', 'Unnamed: 0'], axis = 1)



In [268]:
f_type_test = test['type']
t1t = test['t1']
t2t = test['t2']
t3t = test['t3']
t4t = test['t4']
test = test.drop(['t1','t2', 't3', 't4', 'user', 'type', 'Unnamed: 0'], axis = 1)


In [269]:
print(train.shape)
#print(test.shape)
#print(test.columns)

(6940, 5001)


In [270]:
X_scaled = preprocessing.StandardScaler().fit(train)
train_scaled = X_scaled.transform(train)


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:617: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  


In [271]:
#Xt_scaled = preprocessing.StandardScaler().fit(test)
test_scaled = X_scaled.transform(test)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  


In [272]:
t1 = np.array([t1 == 'E']).astype(int).T
t2 = np.array([t2 == 'N']).astype(int).T
t3 = np.array([t3 == 'F']).astype(int).T
t4 = np.array([t4 == 'P']).astype(int).T


In [273]:
t1t = np.array([t1t == 'E']).astype(int)
t2t = np.array([t2t == 'N']).astype(int)
t3t = np.array([t3t == 'F']).astype(int)
t4t = np.array([t4t == 'P']).astype(int)


In [314]:
def build_model(neurons, input_dim, epochs, momentum):    
    model = Sequential()
    model.add(Dense(neurons, input_dim = input_dim, kernel_constraint=max_norm(2), activation = 'relu'))
    model.add(Dropout(0.6))
    model.add(Dense(neurons, kernel_constraint=max_norm(2),activation = 'relu'))
    model.add(Dropout(0.6))
    model.add(Dense(neurons, kernel_constraint=max_norm(2),activation = 'relu'))
    model.add(Dropout(0.6))  

 #   model.add(Dropout(0.5))
    model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
 #   rms = SGD(lr= learning_rate, decay= learning_rate / epochs, momentum=momentum)
    model.compile(loss='binary_crossentropy', optimizer= 'adam', metrics=["accuracy"])
    return model

In [310]:
batches = 400
validation = 0.2
epochs = 50
neurons = 75
l_rate = 0.00005
model_t1 = build_model(neurons, train.shape[1],  epochs, 0.85)

model_t3 = build_model(neurons, train.shape[1],  epochs, 0.85)



In [323]:
callbacks1 = [EarlyStopping(monitor='val_loss', patience=1),
             ModelCheckpoint(filepath='best_model_t1.h5', monitor='val_loss', save_best_only=True)]
callbacks2 = [EarlyStopping(monitor='val_loss', patience=1),
             ModelCheckpoint(filepath='best_model_t2.h5', monitor='val_loss', save_best_only=True)]
callbacks3 = [EarlyStopping(monitor='val_loss', patience=1),
             ModelCheckpoint(filepath='best_model_t3.h5', monitor='val_loss', save_best_only=True)]
callbacks4 = [EarlyStopping(monitor='val_loss', patience=1),
             ModelCheckpoint(filepath='best_model_t4.h5', monitor='val_loss', save_best_only=True)]
print(test_scaled.shape)
print(t1t.shape)

(1735, 5001)
(1, 1735)


In [317]:
trained_t1 = model_t1.fit(x = train_scaled, y = t1, callbacks=callbacks1,epochs = epochs, batch_size = 200, verbose = False,  validation_split= validation)
eval_t1 = model_t1.evaluate(x = train_scaled, y = t1)
eval_t1t = model_t1.evaluate(x = test_scaled, y = t1t.T)
print(eval_t1, eval_t1t)

1735/1735 [==============================] - 1s 337us/step
[0.5731053141077245, 0.7694524495677233] [0.5997471477181492, 0.7700288184781583]


In [318]:
model_t2 = build_model(neurons, train.shape[1],  epochs, 0.85)
trained_t2 = model_t2.fit(x = train_scaled, y = t2, callbacks=callbacks2,epochs = epochs, batch_size = 200,  validation_split= validation)
eval_t2 = model_t2.evaluate(x = train_scaled, y = t2)
eval_t2t = model_t2.evaluate(x = test_scaled, y = t2t.T)
print(eval_t2, eval_t2t)

Train on 5552 samples, validate on 1388 samples
Epoch 1/50
5552/5552 [==============================] - 8s 1ms/step - loss: 0.5283 - acc: 0.7821 - val_loss: 0.3912 - val_acc: 0.8718
Epoch 2/50
1735/1735 [==============================] - 1s 351us/step
[0.3942062158577724, 0.8609510086111789] [0.4135435838177156, 0.8662824207492795]


In [319]:
trained_t3 = model_t3.fit(x = train_scaled, y = t3, callbacks=callbacks3,epochs = epochs, batch_size = 200,  validation_split= validation)
eval_t3 = model_t3.evaluate(x = train_scaled, y = t3)
eval_t3t = model_t3.evaluate(x = test_scaled, y = t3t.T)
print(eval_t3, eval_t3t)

Train on 5552 samples, validate on 1388 samples
Epoch 1/50
5552/5552 [==============================] - 8s 1ms/step - loss: 0.7417 - acc: 0.5184 - val_loss: 0.6727 - val_acc: 0.5944
Epoch 2/50
5552/5552 [==============================] - 1s 266us/step - loss: 0.6948 - acc: 0.5596 - val_loss: 0.6525 - val_acc: 0.6571
Epoch 3/50
5552/5552 [==============================] - 1s 259us/step - loss: 0.6322 - acc: 0.6286 - val_loss: 0.5941 - val_acc: 0.7313
Epoch 4/50
5552/5552 [==============================] - 1s 259us/step - loss: 0.5615 - acc: 0.7064 - val_loss: 0.5165 - val_acc: 0.7594
Epoch 5/50
5552/5552 [==============================] - 1s 255us/step - loss: 0.4617 - acc: 0.7810 - val_loss: 0.4793 - val_acc: 0.7716
Epoch 6/50
5552/5552 [==============================] - 1s 254us/step - loss: 0.3852 - acc: 0.8285 - val_loss: 0.4785 - val_acc: 0.7781
Epoch 7/50
1735/1735 [==============================] - 1s 366us/step
[0.21547009845288412, 0.9236311238505999] [0.4874519396584041, 0.786

In [324]:
model_t4 = build_model(neurons, train.shape[1],  epochs, 0.85)
trained_t4 = model_t4.fit(x = train_scaled, y = t4,callbacks=callbacks4, epochs = epochs, batch_size = 200,  validation_split= validation)
eval_t4 = model_t4.evaluate(x = train_scaled, y = t4)
eval_t4t = model_t4.evaluate(x = test_scaled, y = t4t.T)
print(eval_t4, eval_t4t)

Train on 5552 samples, validate on 1388 samples
Epoch 1/50
5552/5552 [==============================] - 8s 1ms/step - loss: 0.7131 - acc: 0.5600 - val_loss: 0.6780 - val_acc: 0.6081
Epoch 2/50
5552/5552 [==============================] - 1s 264us/step - loss: 0.6884 - acc: 0.5740 - val_loss: 0.6756 - val_acc: 0.6059
Epoch 3/50
5552/5552 [==============================] - 1s 257us/step - loss: 0.6670 - acc: 0.5947 - val_loss: 0.6730 - val_acc: 0.6073
Epoch 4/50
5552/5552 [==============================] - 1s 262us/step - loss: 0.6511 - acc: 0.6124 - val_loss: 0.6651 - val_acc: 0.6182
Epoch 5/50
5552/5552 [==============================] - 1s 268us/step - loss: 0.6259 - acc: 0.6284 - val_loss: 0.6557 - val_acc: 0.6362
Epoch 6/50
5552/5552 [==============================] - 1s 263us/step - loss: 0.5919 - acc: 0.6639 - val_loss: 0.6412 - val_acc: 0.6434
Epoch 7/50
5552/5552 [==============================] - 1s 263us/step - loss: 0.5311 - acc: 0.7228 - val_loss: 0.6394 - val_acc: 0.6405
Ep

In [325]:
print(eval_t1t[1], eval_t2t[1], eval_t3t[1], eval_t4t[1])

0.7700288184781583 0.8662824207492795 0.7867435159188526 0.6489913545183902


In [23]:
print(pred[1:25])

[[0.28091612]
 [0.2823671 ]
 [0.34586498]
 [0.2656325 ]
 [0.24388891]
 [0.30145082]
 [0.2798758 ]
 [0.3300843 ]
 [0.3149894 ]
 [0.35464972]
 [0.29348773]
 [0.27654028]
 [0.29671323]
 [0.29648384]
 [0.336347  ]
 [0.27972227]
 [0.389725  ]
 [0.35235772]
 [0.43924776]
 [0.31885964]
 [0.3020974 ]
 [0.28448126]
 [0.32630682]
 [0.29284257]]


In [24]:

np.sum(a) / (np.sum(b) + np.sum(a))

NameError: name 'a' is not defined

In [327]:
from sklearn.ensemble import RandomForestClassifier

In [328]:
rft1 = RandomForestClassifier()
rft2 = RandomForestClassifier()
rft3 = RandomForestClassifier()
rft4 = RandomForestClassifier()
rft5 = RandomForestClassifier()

#rft1 = LogisticRegression(C=0.005)
#rft2 = LogisticRegression(C=0.005)
#rft3 = LogisticRegression(C=0.005)
#rft4 = LogisticRegression(C=0.005)
#rft5 = LogisticRegression(C=0.005)


In [329]:
rft1.fit(train_scaled, t1)
rft2.fit(train_scaled, t2)
rft3.fit(train_scaled, t3)
rft4.fit(train_scaled, t4)
rft5.fit(train_scaled, f_type)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """Entry point for launching an IPython kernel.
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  
C:\ProgramData\Anaconda3\lib\site-packa

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [75]:
scoret1 = rft1.score(train_scaled, t1)
scoret2 = rft2.score(train_scaled, t2)
scoret3 = rft3.score(train_scaled, t3)
scoret4 = rft4.score(train_scaled, t4)
scoret5 = rft5.score(train_scaled, f_type)
print(scoret1, scoret2, scoret3, scoret4, scoret5)

0.9910662824207492 0.9979827089337175 0.9926512968299712 0.9762247838616714 0.996685878962536


In [76]:
scoret1 = rft1.score(test_scaled, t1t.T)
scoret2 = rft2.score(test_scaled, t2t.T)
scoret3 = rft3.score(test_scaled, t3t.T)
scoret4 = rft4.score(test_scaled, t4t.T)
scoret5 = rft5.score(test_scaled, f_type_test)
print(scoret1, scoret2, scoret3, scoret4, scoret5)


0.729106628242075 0.8109510086455332 0.7694524495677233 0.6305475504322766 0.33602305475504324


In [326]:
print(rft1.predict(test_scaled[1:250]))
print(rft2.predict(test_scaled[1:550]))
print(rft3.predict(test_scaled[1:250]))
print(rft4.predict(test_scaled[1:250]))
print(scoret1, scoret2, scoret3, scoret4, scoret5)

NameError: name 'rft1' is not defined

In [32]:
keras.

0.23323548496740104